# Asylum Seekers' Profiles Visualization

## Load Libraries

In [1]:
#Load libraries to read docx files
import glob
import zipfile
import xml.etree.ElementTree

WORD_NAMESPACE = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}'
PARA = WORD_NAMESPACE + 'p'
TEXT = WORD_NAMESPACE + 't'
TABLE = WORD_NAMESPACE + 'tbl'
ROW = WORD_NAMESPACE + 'tr'
CELL = WORD_NAMESPACE + 'tc'

#Load libraries to create tables from data
import pandas as pd
import openpyxl

#Load NLP libraries for text analysis
import spacy
nlp = spacy.load('de_core_news_lg')

#Load utility libraries
import re
import dateutil.parser

## Load Data

In [2]:
# Read all files from data folder into memory
path = 'data/'
files = []
for file in glob.glob(path + '*.docx'):
    files.append(file)

with zipfile.ZipFile(files[0]) as docx:
        tree = xml.etree.ElementTree.XML(docx.read('word/document.xml'))

#Since not all documents are structured the same, we will read the necessary
#tables from the first document and remember the first column. This column
#contains descriptions which will be the header of our table

relevant_tables = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

tables = []
for table in tree.iter(TABLE):
    tables.append(table)

table_header = []
for index in relevant_tables:
    table = tables[index]
    for row in table.iter(ROW):
        cells = []
        for cell in row.iter(CELL):
            cells.append(cell)
        table_header.append(''.join(node.text for node in cells[0].iter(TEXT)).replace(':', ''))

table_contents = []

# We now iterate over all documents and look for entries which match the descriptions from the first file
for file in files:
    with zipfile.ZipFile(file) as docx:
        tree = xml.etree.ElementTree.XML(docx.read('word/document.xml'))
    
    tables = []
    for table in tree.iter(TABLE):
        tables.append(table)
    
    table_content = []
    for header in table_header:
        missing = True
        for table in tables:
            for row in table.iter(ROW):
                cells = []
                for cell in row.iter(CELL):
                    cells.append(''.join(node.text for node in cell.iter(TEXT)).replace(':', ''))
                if (cells[0].replace('-','').replace(' ', '') == header.replace('-','').replace(' ', '')) & (len(cells) > 1) & (missing):
                    missing = False
                    table_content.append(cells[1])
        if missing:
            table_content.append('-')
    
    # We add the name of the document as last entry of the row
    table_content.append(file)
    table_contents.append(table_content)

# Add header entry for name of the document
table_header.append('Dokument')

# Put all data into Pandas DataFrame
df = pd.DataFrame.from_records(table_contents, columns=table_header)

In [3]:
df.describe()

,Name/VornameAdresseTelefonnummer(n)/ ErreichbarkeitE-Mail-Adresse(n),Staatsangehörigkeit,Geburtsdatum und -ort,Erstsprache(n),Aufenthaltsstatus,Einreise in die Schweiz,Zivilstand,"Kinder (Anzahl, Alter)",AHV-Nr.,"OrganisationName, E-Mail und Tel. der zuständigen Person",...,Berufserfahrung,"Arbeitserfah- rung generell (ausserberufli- che Tätigkeiten, Integrations-/ Beschäftigungs- massnahmen, Freiwilligenar- beit etc.)",Gesundheit,Kurzzusammen- fassung der Si- tuation (Ist-Zu- stand),Chancen,Hindernisse,Ziele für weitere Integrationspla- nung,"Bedarf für ver- tiefte Abklärun- gen/Ziele→Instrumente und Methoden siehe Formu- lare/Doku- mente""Kompe- tenzerfassung"", ""Praxisassess-ment""","Nächste Schritte, Sofort- massnahmen",Dokument
count,72,72,72,72,72,72,72,72,72,72,...,72,72,72,72,72,72,72,72,72,72
unique,7,20,63,29,25,46,11,34,3,19,...,54,41,59,44,41,41,41,2,52,72
top,,Syrien,"01.04.1992 , Damaskus",Arabisch,,,ledig,keine,,,...,,,,,,,,-,,data\Kurzassessment_ZH_18.docx
freq,64,18,3,17,11,17,19,10,59,49,...,7,20,3,20,19,25,21,69,16,1


In [4]:
df.head()

,Name/VornameAdresseTelefonnummer(n)/ ErreichbarkeitE-Mail-Adresse(n),Staatsangehörigkeit,Geburtsdatum und -ort,Erstsprache(n),Aufenthaltsstatus,Einreise in die Schweiz,Zivilstand,"Kinder (Anzahl, Alter)",AHV-Nr.,"OrganisationName, E-Mail und Tel. der zuständigen Person",...,Berufserfahrung,"Arbeitserfah- rung generell (ausserberufli- che Tätigkeiten, Integrations-/ Beschäftigungs- massnahmen, Freiwilligenar- beit etc.)",Gesundheit,Kurzzusammen- fassung der Si- tuation (Ist-Zu- stand),Chancen,Hindernisse,Ziele für weitere Integrationspla- nung,"Bedarf für ver- tiefte Abklärun- gen/Ziele→Instrumente und Methoden siehe Formu- lare/Doku- mente""Kompe- tenzerfassung"", ""Praxisassess-ment""","Nächste Schritte, Sofort- massnahmen",Dokument
0,Siehe Lebenslauf,Russland / Schweiz,LL,Russisch,CH,2006,getrennt,"1, 11 Jahre",-,,...,Siehe LL.Hat während Studium in Bank gearbeite...,,i.o.,"Schätzt ihre Situation sehr gut ein, sehr lern...",Genau abklären!,SpracheLange Mutterschaftspause Veraltetes Wissen,,,,data\Kurzassesment_BE_15.docx
1,,Afghanistan,01.01.1985,Persisch,FVA,12.12.2015,verheiratet,3,,,...,"Im Herkunftsland ca. 19 Jahre als Bodenleger, ...",-Seit 26.09.2017 bis heute beim Betriebscenter...,Klient hat immer wieder Zahnschmerzen und war ...,KL ist sehr motiviert und hat Durchhaltevermög...,"-freundlich, offen-A1 Niveau erreicht-2 Jahre ...",-Sorgen um die Eltern und seine Familie-Teilwe...,Abklärung der geeignetsten Möglichkeiten für d...,-,,data\Kurzassesment_BE_16.docx
2,,Syrien,"05.09.1988, Al Raqqa",Arabisch,F VA AS,16.10.2015,Ledig,Keine,,,...,dito,dito,"Im dritten Monat schwanger, ab und zu schwindlig",,,,,-,Zurzeit pendent,data\Kurzassessment_AG_01.docx
3,,Eritrea,01.01.1996,Tigrinya,F VA AS,05.09.2015,ledig,"2 (2015, 2018)",,,...,Keine Ausbildung; jedoch im Wald Gold geschürf...,,Gesundheitlich gehe es der Klientin gut. Sie k...,,,,,-,Anmeldung Alphakurs bei ECAP mit Kinderbetreu...,data\Kurzassessment_AG_02.docx
4,,Eritrea,01.06.1990,Tigrinya,B-Flüchtling,27.07.2016,ledig,keine,,,...,Landwirtschaft Familienbetrieb; EritreaMetallb...,Asylantenarbeitsprojekt Werkhof * (Arbeitsbest...,Zum Zeitpunkt des Gesprächs Schmerzen im recht...,,,,,-,A2 Deutschkurs * Montag bis Freitag NM 3 Monat...,data\Kurzassessment_AG_03.docx


## Cleanse/Transform Data

In [5]:
# The column "Geburtsdatum und -ort" contains either the birthdate or birthdate and birthplace
# We remove everything except the birthdate
def get_date(string):
    res = None
    string2 = re.sub("[^0-9.:]", "", string)
    try:
        res = dateutil.parser.parse(string2)
        return res
    except:
        return('-')

# For the prototype we want some dates formatet as strings
def date_to_string(date):
    try:
        return date.strftime('%d.%m.%Y')
    except:
        return '-'

# The end of the constituency is (for simplicity) calculated by adding 7 years to the imigration date
def add_year(date):
    try:
        return date + datetime.timedelta(days=7*365)
    except:
        return '-'
    
df['Geburtsdatum und -ort'] = df['Geburtsdatum und -ort'].apply(get_date)
df['Geburtsdatum'] = df['Geburtsdatum und -ort'].apply(date_to_string)
df['Einreise in die Schweiz'] = df['Einreise in die Schweiz'].apply(get_date)
df['Ende Zuständigkeit'] = df['Einreise in die Schweiz'].apply(add_year)
df['Einreise'] = df['Einreise in die Schweiz'].apply(date_to_string)
df['Ende Zuständigkeit'] = df['Ende Zuständigkeit'].apply(date_to_string)
df['Dokument'] = df['Dokument'].apply(lambda x: x.replace('.docx', '').replace('data\\', ''))

## Try some NLP

In [6]:
def get_entities(string):
    entities = []
    
    if string is not None:
        doc = nlp(string)
        for ent in doc.ents:
            #if ent.label_ != 'LOC':
            #    entities.append(ent.text)
            entities.append(ent.text)
            
    return entities
        
df['Massnahmen_Entitäten'] = df['Nächste Schritte, Sofort- massnahmen'].apply(get_entities)
df['Arbeitserfahrung'] = df['Arbeitserfah- rung generell (ausserberufli- che Tätigkeiten, Integrations-/ Beschäftigungs- massnahmen, Freiwilligenar- beit etc.)'].apply(get_entities)

## Export part of the Table as Json for usage in React App

In [7]:
df_tmp = df[['Dokument', 'Staatsangehörigkeit', 'Aufenthaltsstatus', 'Geburtsdatum', 'Erstsprache(n)', 'Lokale Amts- sprache', 'Einreise', 'Zivilstand', 'Massnahmen_Entitäten', 'Arbeitserfahrung', 'Kurzzusammen- fassung der Si- tuation (Ist-Zu- stand)', 'Verfügbarkeit', 'Nächste Schritte, Sofort- massnahmen', 'Kinder (Anzahl, Alter)', 'Ende Zuständigkeit']].copy()
df_tmp.reset_index()
#df_tmp.to_excel('output.xlsx')
df_tmp.to_json ('data.json', orient='records')